In [2]:
import pandas as pd
import numpy as np
import catboost
import plotly.express as px

# Read Data

In [3]:
df_credits = pd.read_csv('../data/User Credits Student Access.csv', encoding='utf-8')
df_atlas = pd.read_csv('../data/Atlas Cechu Student Access.csv', encoding='utf-8')
df_payments = pd.read_csv('../data/Payments Student Access.csv', encoding='utf-8')

# Negative Values investigation & Chekcing other things

In [4]:
df_c_negative = df_credits[df_credits['credits']<0]
df_credits[df_credits.user.isin(df_c_negative.user) == True]

# IDK what I'm doing with payments
df_payments[df_payments.user.isin(df_c_negative.user)==True].sort_values(['user','created_at'])
df_payments.state.unique()
df_payments[df_payments.user=='STUD54678']
df_credits[df_credits.user=='STUD54678']

,user,credits,is_active,is_verified,is_locked,wage
89906,STUD54678,789,1,1,0,Peníze


In [5]:
df_payments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30087 entries, 0 to 30086
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          30086 non-null  float64
 1   created_at  30087 non-null  object 
 2   changed_at  30087 non-null  object 
 3   user        27652 non-null  object 
 4   batch       27502 non-null  object 
 5   credits     30086 non-null  float64
 6   state       30086 non-null  object 
 7   type        30086 non-null  object 
dtypes: float64(2), object(6)
memory usage: 1.8+ MB


# Data Cleaning

In [6]:
df_credits_cleaned = df_credits[df_credits['credits']>0]
df_payments_cleaned = df_payments[df_payments['id'].notna()]

# Jaká tabulka bude základní pro merge? - Payments

In [7]:
# Convert the boolean series to a DataFrame with a column name
atlas_in_credits = df_credits_cleaned.user.isin(df_atlas.user_id).rename('User in Atlas')
atlas_in_payments = df_payments_cleaned.user.isin(df_atlas.user_id)

# Plotting
fig_ac = px.bar(atlas_in_credits.value_counts(), title='How many users from Atlas Cechu are in the credits table', labels={'index': 'User in Atlas (True/False)', 'value': 'Count'})
fig_ap = px.bar(atlas_in_payments.value_counts(), title='How many users from Atlas Cechu are in the payments table', labels={'index': 'User in Atlas (True/False)', 'value': 'Count'})

fig_ac.show()
fig_ap.show()

In [8]:
df_atlas.user_id.isin(df_credits_cleaned.user).value_counts()


user_id
True     40672
False     2861
Name: count, dtype: int64

In [9]:
df_atlas.user_id.isin(df_payments_cleaned.user).value_counts()

user_id
False    34824
True      8709
Name: count, dtype: int64

Since we want to find out when who pulls what, it makes sense to use the payments table and connect data to it - even though there are users in payments tables that are not in Atlas Cechu

In [10]:
df_merge_credits = pd.merge(df_payments_cleaned, df_credits_cleaned, how="left", left_on='user', right_on='user')
df_merge_credits[df_merge_credits.user=='STUD92308']

,id,created_at,changed_at,user,batch,credits_x,state,type,credits_y,is_active,is_verified,is_locked,wage
18722,11394.0,2023-01-10 08:38:38.329030+00:00,2024-01-10 17:27:33.572797+00:00,STUD92308,155,605.0,PAID,MONEY,295.0,1.0,1.0,0.0,Peníze
22495,7621.0,2021-12-21 10:00:03.873926+00:00,2024-01-10 17:28:50.738015+00:00,STUD92308,121,2223.0,PAID,MONEY,295.0,1.0,1.0,0.0,Peníze
28483,1633.0,2019-06-20 04:45:33.065690+00:00,2024-01-10 17:30:41.662759+00:00,STUD92308,46,502.0,PAID,MONEY,295.0,1.0,1.0,0.0,Peníze
30084,31.0,2017-08-10 21:58:35.586238+00:00,2024-01-10 17:31:11.867765+00:00,STUD92308,NaN,511.0,PAID,MEDICINS_SANS_FRONTIERS,295.0,1.0,1.0,0.0,Peníze


In [11]:
df_merge_full = pd.merge(df_merge_credits, df_atlas, how='left', left_on='user', right_on='user_id')

In [12]:
df_merge_full.to_csv('../data_output/df_merge_full.csv')

# Modeling

In [13]:
import catboost
from sklearn.model_selection import train_test_split
